# display_plot

> Plotting and animating stroke data using matplotlib.

In [ ]:
#| default_exp stroke3.display_plot

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import matplotlib.pyplot as plt


def plot_strokes(strokes, target_size=200, lw=2):
    fig = plt.figure()
    ax = plt.axes(xlim=(0, target_size + 2 * lw), ylim=(-target_size - 2 * lw, 0))
    ax.set_facecolor("white")

    # remove the axis
    ax.grid = False
    ax.set_xticks([])
    ax.set_yticks([])

    lines = []
    for s in strokes:
        (line,) = ax.plot([], [], lw=lw)
        line.set_data(s[:, 0], -s[:, 1])
        lines.append(line)
    plt.show()

In [ ]:
#| export
from lxml import etree


def parse_svg(input_file):
    """Parse the SVG XML, to allow extracting individual elements"""
    parsed = etree.parse(input_file)
    return parsed.getroot()

In [ ]:
#| export
import numpy as np
from matplotlib import animation


def create_animation(
    strokes, fname="video.mp4", fps=60, target_size=200, lw=2, trailing_frames=30
):
    seq_length = np.vstack(strokes).shape[0]
    print(seq_length)

    i = 0
    j = 0

    # First set up the figure, the axis, and the plot element we want to animate
    fig = plt.figure()
    ax = plt.axes(xlim=(0, target_size + 2 * lw), ylim=(-target_size - 2 * lw, 0))
    ax.set_facecolor("white")
    (line,) = ax.plot([], [], lw=lw)

    # remove the axis
    ax.grid = False
    ax.set_xticks([])
    ax.set_yticks([])

    # initialization function: plot the background of each frame
    def init():
        line.set_data([], [])
        return (line,)

    # animation function.  This is called sequentially
    def animate(frame):
        nonlocal i, j, line
        if i < len(strokes):
            x = strokes[i][:, 0]
            y = strokes[i][:, 1]
            line.set_data(x[0:j], -y[0:j])

            if j >= len(x):
                i += 1
                j = 0
                (line,) = ax.plot([], [], lw=lw)
            else:
                j += 1
        else:
            # if i has already incremented past all the strokes,
            # that means this is a "trailing frame" (meant to leave
            # the finished drawing onscreen for a moment).
            pass
        return (line,)

    # call the animator.  blit=True means only re-draw the parts that have changed.
    total_frames = seq_length + len(strokes) + trailing_frames
    anim = animation.FuncAnimation(
        fig, animate, init_func=init, frames=total_frames, blit=True
    )
    plt.close()

    # save the animation as an mp4.
    anim.save(fname, fps=fps, extra_args=["-vcodec", "libx264"])

In [ ]:
#| export
import base64
import io

from IPython.display import HTML, display


def show_video(fname="video.mp4"):
    """
    create_animation(strokes, fname="video.mp4", lw=2)
    show_video("video.mp4")
    """
    video = io.open(fname, "r+b").read()
    encoded = base64.b64encode(video)
    html_data = f"""<video alt="video" autoplay loop>
                    <source src="data:video/mp4;base64,{encoded.decode('ascii')}" type="video/mp4" />
                </video>"""
    display(HTML(data=html_data))

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()